In [ ]:
import numpy as np
import pandas as pd
import multiprocessing
from matplotlib import pyplot as plt

In [ ]:
name = "sample"

print('Reading', name+'_MCP.csv', '...')
%time MCP_original_dataframe = pd.read_csv(name+'_MCP.csv')
print('done.')
print()
print('Reading', name+'_CsI.csv', '...')
%time CsI_original_dataframe = pd.read_csv(name+'_CsI.csv')
print('done.')
print()
print('Reading', name+'_MWPC.csv', '...')
%time MWPC_original_dataframe = pd.read_csv(name+'_MWPC.csv')
print('done.')

In [ ]:
# (nondimensionalized) raw data of each detector in ndarray form.
MCP_raw_data  = MCP_original_dataframe.values
CsI_raw_data  = CsI_original_dataframe.values
MWPC_raw_data = MWPC_original_dataframe.values

# some constants.
c     = 299.79                # c         (mm/ns)
mc2   = 0.511                 # mc^2      (MeV)
B     = 0.1                   # B_MagSpec (T)
mc_eB = mc2 / (B * c) * 1e3   # mc_eB     (mm)
m_eB  = mc_eB / c             # m_eB      (ns)
# experiment parameters.
l_acc   = 400                               / mc_eB
eU_acc  = 7e-3                              / mc2
l_trans = 2050                              / mc_eB
r_turn  = 350                               / mc_eB
# derived parameters.
v_acc   = np.sqrt(2 * eU_acc) #np.sqrt(1 - 1 / np.square(eU_acc + 1))
TOF_fix = l_acc * np.sqrt(2 / eU_acc) + l_trans / v_acc #l_acc * np.sqrt(1 + 2 / eU_acc) + l_trans / v_acc

# nondimensionalize t(ns).
MCP_raw_data.T[1]    = MCP_raw_data.T[1]    / m_eB
# nondimensionalize x,y(mm).
MCP_raw_data.T[2:4]  = MCP_raw_data.T[2:4]  / mc_eB

# nondimensionalize t(ns).
CsI_raw_data.T[1]    = CsI_raw_data.T[1]    / m_eB
# nondimensionalize E(MeV).
CsI_raw_data.T[2]    = CsI_raw_data.T[2]    / mc2

# nondimensionalize t(ns).
MWPC_raw_data.T[1]   = MWPC_raw_data.T[1]   / m_eB
# nondimensionalize x,y,z(mm).
MWPC_raw_data.T[2:5] = MWPC_raw_data.T[2:5] / mc_eB

print('mc^2    =', mc2, 'MeV')
print('mc/eB   =', mc_eB, 'mm')
print('m/eB    =', m_eB, 'ns')
print('B_MS    =', B, 'T')
print('l_acc   =', l_acc * mc_eB, 'mm')
print('l_trans =', l_trans * mc_eB, 'mm')
print('r_turn  =', r_turn * mc_eB, 'mm')
print('v_acc   =', v_acc, 'c')

# some visualization.
label_fontsize = 16
ticks_fontsize = 14
# MCP
h2d = plt.hist2d(MCP_raw_data.T[2] * mc_eB, MCP_raw_data.T[3] * mc_eB,
                 range=((-30, 30), (-30, 30)), bins=100, cmap='RdBu_r', cmin=1)
plt.xlabel(r'$x_{hit,MCP}\ [\rm{mm}]$', fontsize=label_fontsize)
plt.ylabel(r'$y_{hit,MCP}\ [\rm{mm}]$', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
cbar = plt.colorbar(h2d[3])
cbar.ax.tick_params(labelsize=ticks_fontsize)
plt.show()
# CSI
plt.hist(CsI_raw_data.T[2] * mc2*1e3, range=(509, 512), bins=50)
plt.xlabel(r'$E_{CsI}\ [keV]$', fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

In [ ]:
MWPC_hit_count = MWPC_original_dataframe.SN.value_counts()
# visualizing MWPC hit count.
label_fontsize = 16
ticks_fontsize = 14
hit_plot_max = 20
plt.hist(MWPC_hit_count, range=(1, hit_plot_max + 1), bins=hit_plot_max)
plt.xlabel('MWPC hit count', fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(range(1, hit_plot_max + 2, 2) ,fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

# set up filter rules.
# filter event that hit 5 MWPCs.
# ToDo: also select hit CsI.
#       pattern detection. (mu->3e+2v required.)
filtered_sn = MWPC_hit_count[MWPC_hit_count == 5].index.sort_values()

print('Raw data consists of', MCP_original_dataframe.shape[0], 'events.')
print('After filtering,', filtered_sn.shape[0], 'events will be retained.')
print('Dropped ratio =', (1 - filtered_sn.shape[0] / MCP_original_dataframe.shape[0]) * 100, '%')

In [ ]:
# return filtered raw data in ndarray form.
# Note:
# The first subscript of this ndarray points to the event(0~n),
# The second subscript points to SN(0) and the data of the three detectors MCP, CsI, and MWPC in this event(1~3),
# The third subscript points to one of the records of the detector in this event 
# (Here, there should be only one record for MCP and CsI, while MWPC can have several records).
# The fourth subscript points to the specific value of this record, which corresponds to each column in the original data. 
def ConstructRawDataArray(filtered_sn, MCP_raw_data, CsI_raw_data, MWPC_raw_data):
    effective_event_num = filtered_sn.shape[0]
    # the raw data array.
    raw_data = np.ndarray((effective_event_num, 4), dtype=object)
    # fill SN.
    raw_data.T[0] = filtered_sn
    # fill MCP, CsI, MWPC data.
    MCP_iter  = 0
    CsI_iter  = 0
    MWPC_iter = 0
    for i in range(effective_event_num):
        # fill MCP data of this event.
        while MCP_iter < MCP_raw_data.shape[0]: 
            if MCP_raw_data[MCP_iter][0] != filtered_sn[i]: MCP_iter += 1
            else: break
        if MCP_iter < MCP_raw_data.shape[0]:
            raw_data[i][1] = MCP_raw_data[MCP_iter][1:4]
        # fill CsI data of this event.
        CsI_this_event_list = list()
        while CsI_iter < CsI_raw_data.shape[0]: 
            if CsI_raw_data[CsI_iter][0] != filtered_sn[i]: CsI_iter += 1
            else: break
        while CsI_iter < CsI_raw_data.shape[0]: 
            if CsI_raw_data[CsI_iter][0] == filtered_sn[i]:
                CsI_this_event_list.append(CsI_raw_data[CsI_iter][1:3])
                CsI_iter += 1
            else: break
        raw_data[i][2] = np.array(CsI_this_event_list)
        # fill MWPC data of this event.
        MWPC_this_event_list = list()
        while MWPC_iter < MWPC_raw_data.shape[0]: 
            if MWPC_raw_data[MWPC_iter][0] != filtered_sn[i]: MWPC_iter+=1
            else: break
        while MWPC_iter < MWPC_raw_data.shape[0]: 
            if MWPC_raw_data[MWPC_iter][0] == filtered_sn[i]:
                MWPC_this_event_list.append(MWPC_raw_data[MWPC_iter][1:5])
                MWPC_iter += 1
            else: break
        raw_data[i][3] = np.array(MWPC_this_event_list)
    return raw_data

print('Filtering raw data ...')
%time raw_data = ConstructRawDataArray(filtered_sn, MCP_raw_data, CsI_raw_data, MWPC_raw_data)
print('done.')

In [ ]:
# paras:
# x = [x1, x2, x3, ..., xn]
# y = [y1, y2, y3, ..., yn]
#
# returns:
# xc, yc: Center of circle.
# R     : Radius of circle.
def FitCircle(x, y, verbose=False):
    x_mean = x.mean()
    y_mean = y.mean()
    u = x - x_mean
    v = y - y_mean

    u2 = np.square(u)
    v2 = np.square(v)
    uv = u * v
    
    sum_u2 = u2.sum()
    sum_v2 = v2.sum()
    sum_uv = uv.sum()
    sum_u2v_v3 = (v * (u2 + v2)).sum()
    sum_u3_uv2 = (u * (u2 + v2)).sum()

    det = sum_u2 * sum_v2 - sum_uv * sum_uv
    if det == 0:
        if verbose:
            print('(x_i, y_i) are on a straight line, or completely overlap. nan will be returned.')
        return (np.nan, np.nan, np.nan)
    
    uc = (sum_v2 * sum_u3_uv2 - sum_uv * sum_u2v_v3) / det
    vc = (sum_u2 * sum_u2v_v3 - sum_uv * sum_u3_uv2) / det
    R = np.sqrt((np.square(u - uc) + np.square(v - vc)).mean())

    return (uc + x_mean, vc + y_mean, R)

# paras:
# arc = [arc1, arc2, arc3, ..., arcn] 
#     = R * [theta1, theta2, theta3, ..., thetan]
# z   = [z1, z2, z3, ..., zn]
#
# returns:
# z0       : z when arc=0. Doesn't make much sense.
# tan_alpha: The direction of momentum in the cylindrical coordinate system. alpha = angle between p and e_theta.
def FitAxial(arc, z, verbose=False):
    arc_mean = arc.mean()
    z_mean = z.mean()
    
    det = np.square(arc).mean() - arc_mean * arc_mean
    if det == 0:
        if verbose:
            print('arc_i are completely overlap, or (arc_i, z_i) are too steep. nan will be returned.')
        return (np.nan, np.nan)

    tan_alpha = ((arc * z).mean() - arc_mean * z_mean) / det
    z0 = z_mean - arc_mean * tan_alpha

    return (z0, tan_alpha)

# data should be nondimensionalized.
def FitHelix(MWPC_event, verbose=False):
    x = MWPC_event.T[1]
    y = MWPC_event.T[2]
    z = MWPC_event.T[3]

    # fit circle (xy prj.).
    xc, yc, R = FitCircle(x, y, verbose)

    # fit z.
    theta = np.zeros(z.shape[0])
    for i in range(theta.shape[0]):
        delta_x = x[i] - xc
        delta_y = y[i] - yc
        if delta_y > 0:
            if delta_x > 0:
                theta[i] = np.arctan(delta_x / delta_y)
            else:
                theta[i] = np.arctan(delta_x / delta_y) + 2 * np.pi
        else:
            theta[i] = np.arctan(delta_x / delta_y) + np.pi
    arc = R * theta

    z0, tan_alpha = FitAxial(arc, z, verbose)

    # scoring.
    x_hat = xc + R * np.sin(theta)
    y_hat = yc + R * np.cos(theta)
    z_hat = z0 + arc * tan_alpha

    error_sq = np.square(x - x_hat) + np.square(y - y_hat) + np.square(z - z_hat)
    mse = error_sq.mean()
    mae = np.sqrt(error_sq).mean()
    mst = (np.square(x - x.mean()) + np.square(y - y.mean()) + np.square(z - z.mean())).mean()
    r2 = 1 - mse / mst

    if verbose:
        # plotting
        label_fontsize = 16
        ticks_fontsize = 14
        # plot x-y
        plt.plot(x_hat, y_hat, 'r+:')
        plt.scatter(x, y)
        plt.axis('equal')
        plt.xticks(fontsize=ticks_fontsize)
        plt.yticks(fontsize=ticks_fontsize)
        plt.xlabel(r'$x\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.ylabel(r'$y\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.show()
        # plot full x y
        theta_plot = np.linspace(0, 6.28, 100)
        plt.plot(xc + R * np.sin(theta_plot), yc + R * np.cos(theta_plot), 'r:')
        plt.scatter(x, y)
        plt.scatter(xc, yc)
        plt.axis('equal')
        plt.xticks(fontsize=ticks_fontsize)
        plt.yticks(fontsize=ticks_fontsize)
        plt.xlabel(r'$x\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.ylabel(r'$y\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.show()
        # plot arc-z
        plt.plot(arc, z_hat, 'r+:')
        plt.scatter(arc, z)
        plt.axis('equal')
        plt.xticks(fontsize=ticks_fontsize)
        plt.yticks(fontsize=ticks_fontsize)
        plt.xlabel(r'$s_{arc}\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.ylabel(r'$z\left/\frac{m_e c}{eB_{MS}}\right.$', fontsize=label_fontsize)
        plt.show()
    
    return [xc, yc, R, np.arctan(tan_alpha), mse, mae, r2]

def AnalyzeMWPCData(raw_event, verbose=False):
    return FitHelix(raw_event[3], verbose)

sample = AnalyzeMWPCData(raw_data[6], True)
print('xc    =', sample[0])
print('yc    =', sample[1])
print('R     =', sample[2])
print('alpha =', sample[3])
print('MSE   =', sample[4])
print('MAE   =', sample[5])
print('R^2   =', sample[6])

In [ ]:
# analyze MWPC data.
print('Analyzing MWPC data ...')
pool = multiprocessing.Pool()
%time MWPC_raw_result = np.array(pool.map(AnalyzeMWPCData, raw_data))
pool.close()
pool.join()
print('done.')

In [ ]:
# set up data filter and evaluate the results of fitting and filtering.

# step0: drop nans.
is_not_nan = ~np.isnan(MWPC_raw_result.T[0])
MWPC_result = MWPC_raw_result[is_not_nan]
analyzable_data = raw_data[is_not_nan]

# filter data based on filter_rules.
#
# step1: R^2 > 0.99
filter_rules = MWPC_result.T[6] > 0.99
MWPC_result = MWPC_result[filter_rules]
analyzable_data = analyzable_data[filter_rules]
# step2: log10(beta*gamma) in 3*sigma
log_beta_gamma_stddev = np.log10(MWPC_result.T[2] / np.cos(MWPC_result.T[3])).std()
log_beta_gamma_mean = np.log10(MWPC_result.T[2] / np.cos(MWPC_result.T[3])).mean()
filter_rules = (log_beta_gamma_mean - 3*log_beta_gamma_stddev < np.log10(MWPC_result.T[2] / np.cos(MWPC_result.T[3]))) & \
               (np.log10(MWPC_result.T[2] / np.cos(MWPC_result.T[3])) < log_beta_gamma_mean + 3*log_beta_gamma_stddev)
MWPC_result = MWPC_result[filter_rules]
analyzable_data = analyzable_data[filter_rules]

print('Among', raw_data.shape[0], 'events, there were', 
      raw_data.shape[0] - analyzable_data.shape[0], 'events are filtered out.')
print('Dropped ratio =', (1 - analyzable_data.shape[0] / raw_data.shape[0]) * 100, '%')

# plot xc, yc, R, alpha, MSE, MAE, R2, evaluate the results of fitting and filtering.
# Note: due to nondimensionalization, R/cos(alpha) is effectively beta*gamma.
label_fontsize = 16
ticks_fontsize = 14
bins_1d = 150
bins_2d = 50

# xc, yc
h2d = plt.hist2d(MWPC_raw_result.T[0] * mc_eB, MWPC_raw_result.T[1] * mc_eB,
                 range=((-200, 200), (-200, 200)), bins=bins_2d, cmap='RdBu_r', cmin=1)
plt.xlabel(r'$x_c\ [\rm{mm}]$', fontsize=label_fontsize)
plt.ylabel(r'$y_c\ [\rm{mm}]$', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
cbar = plt.colorbar(h2d[3])
cbar.ax.tick_params(labelsize=ticks_fontsize)
plt.show()

# beta*gamma
plt.hist(MWPC_result.T[2] / np.cos(MWPC_result.T[3]), range=(0, 500), bins=bins_1d)
plt.xlabel(r'$\beta\gamma$', fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

# log10(beta*gamma)
plt.hist(np.log10(MWPC_result.T[2] / np.cos(MWPC_result.T[3])), range=(0, 4), bins=bins_1d)
plt.xlabel(r'$\lg{(\beta\gamma)}$', fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

# alpha
plt.hist(MWPC_result.T[3] * 360/(2*np.pi), range=(-75, 75), bins=bins_1d)
plt.xlabel(r'$\alpha$', fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

# MSE
plt.hist(MWPC_result.T[4], range=(0, 0.4), bins=bins_1d)
plt.xlabel(r'$MSE\left/\left(\frac{m_e c}{eB_{MS}}\right)^2\right.$'
           r'$\quad\frac{m_e c}{eB_{MS}}=$'+'{:.2f}mm'.format(mc_eB), fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

# MAE
plt.hist(MWPC_result.T[5], range=(0,1), bins=bins_1d)
plt.xlabel(r'$MAE\left/\frac{m_e c}{eB_{MS}}\right.$'
           r'$\quad\frac{m_e c}{eB_{MS}}=$'+'{:.2f}mm'.format(mc_eB), fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

# R2
plt.hist(MWPC_result.T[6], range=(0.99, 1), bins=bins_1d)
plt.xlabel(r'$R^2$', fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

In [ ]:
def FindTOFAndRdca(analyzable_MCP_event_and_corresponding_MWPC_result_and_MWPC_event):
    MCP_event = analyzable_MCP_event_and_corresponding_MWPC_result_and_MWPC_event[0]
    MWPC_result = analyzable_MCP_event_and_corresponding_MWPC_result_and_MWPC_event[1]
    MWPC_event = analyzable_MCP_event_and_corresponding_MWPC_result_and_MWPC_event[2]

    R_dca = np.sqrt(np.square(MCP_event[1] - MWPC_result[0]) + np.square(MCP_event[2] - MWPC_result[1])) - MWPC_result[2]
    TOF   = MCP_event[0] - MWPC_event[0][0]
    TOF_E = TOF_fix + 0.5*np.pi * (r_turn - MCP_event[1]) / v_acc

    return [R_dca, TOF, TOF_E]

sample_index = 14
sample = FindTOFAndRdca([analyzable_data[sample_index][1], MWPC_result[sample_index], analyzable_data[sample_index][3]])
print('R_dca     =', sample[0])
print('TOF       =', sample[1])
print('TOF_E     =', sample[2])
print('TOF-TOF_E =', sample[1] - sample[2])

In [ ]:
analyzable_MCP_data_and_MWPC_result_and_MWPC_data = np.ndarray((analyzable_data.shape[0], 3), dtype=object)
for i in range(analyzable_MCP_data_and_MWPC_result_and_MWPC_data.shape[0]):
    analyzable_MCP_data_and_MWPC_result_and_MWPC_data[i][0] = analyzable_data[i][1]
    analyzable_MCP_data_and_MWPC_result_and_MWPC_data[i][1] = MWPC_result[i]
    analyzable_MCP_data_and_MWPC_result_and_MWPC_data[i][2] = analyzable_data[i][3]

pool = multiprocessing.Pool()
%time muonium_result = np.array(pool.map(FindTOFAndRdca, analyzable_MCP_data_and_MWPC_result_and_MWPC_data))
pool.close()
pool.join()

In [ ]:
# plot Rdca, TOF, TOF_E
label_fontsize = 16
ticks_fontsize = 14
bins_1d = 100
bins_2d = 50

# Rdca 
plt.hist(muonium_result.T[0] * mc_eB, range=(-50, 50), bins=bins_1d)
plt.xlabel(r'$R_{dca}\ [\rm{mm}]$', fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

# TOF
plt.hist(muonium_result.T[1] * m_eB, range=(65, 72), bins=bins_1d)
plt.xlabel(r'$TOF\ [\rm{ns}]$', fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

# TOF
plt.hist(muonium_result.T[2] * m_eB, range=(65, 72), bins=bins_1d)
plt.xlabel(r'$TOF_{expected}\ [\rm{ns}]$', fontsize=label_fontsize)
plt.ylabel('count', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
plt.show()

# Rdca, TOF-TOF_E
h2d = plt.hist2d((muonium_result.T[1] - muonium_result.T[2]) * m_eB, muonium_result.T[0] * mc_eB,
                 range=((-10, 10), (-50, 50)), bins=bins_2d, cmap='RdBu_r', cmin=1)
plt.xlabel(r'$TOF - TOF_{expected}\ [\rm{ns}]$', fontsize=label_fontsize)
plt.ylabel(r'$R_{dca}\ [\rm{mm}]$', fontsize=label_fontsize)
plt.xticks(fontsize=ticks_fontsize)
plt.yticks(fontsize=ticks_fontsize)
cbar = plt.colorbar(h2d[3])
cbar.ax.tick_params(labelsize=ticks_fontsize)
plt.show()